### Базовые библиотеки

In [308]:
import pandas as pd
import numpy as np
import datetime
import random
import warnings
warnings.simplefilter('ignore')
import xgboost as xgb
from random import shuffle
from collections import Counter
import nltk
import re
import math


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20)

data_dir = ''

### Загрузка данных

In [349]:
train = pd.read_csv(data_dir + 'train.csv',sep='\t') 
test = pd.read_csv(data_dir + 'test.csv',sep='\t') 

In [5]:
train.shape

(728, 7)

### Вот так они выглядят

In [6]:
train

,text,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6
0,. Об утверждении годового отчета ОАО « НЛМК » ...,0,0,0,1,0,0
1,2 . Об избрании членов Совета директоров ОАО «...,0,0,0,0,1,0
2,3 . Об избрании Президента ( Председателя Прав...,0,0,1,0,0,0
3,4 . Об избрании членов Ревизионной комиссии ОА...,0,0,0,1,0,0
4,5 . Об утверждении Аудитора ОАО « НЛМК » Предл...,0,0,0,1,0,0
5,"6 . Об одобрении сделок , в совершении которых...",0,0,0,1,0,0
6,7 . О выплате вознаграждений членам Совета дир...,0,0,0,1,0,0
7,"первому вопросу повестки дня : Число голосов ,...",0,0,0,0,1,0
8,По второму вопросу повестки дня : Число голосо...,0,0,0,1,0,0
9,По третьему вопросу повестки дня : Число голос...,0,0,0,1,0,0


### Частотность лейблов:

In [7]:
np.sum(train[['Label_1', 'Label_2', 'Label_3', 'Label_4', 'Label_5', 'Label_6']], axis=0)

Label_1      6
Label_2      8
Label_3     14
Label_4    601
Label_5     84
Label_6     24
dtype: int64

In [8]:
np.sum(train[['Label_1', 'Label_2', 'Label_3', 'Label_4', 'Label_5', 'Label_6']], axis=0)/train.shape[0]

Label_1    0.008242
Label_2    0.010989
Label_3    0.019231
Label_4    0.825549
Label_5    0.115385
Label_6    0.032967
dtype: float64

### Количество записей, у кого есть одновременно 0,1,2 и 3 лейбла

In [9]:
Counter(np.sum(train[['Label_1', 'Label_2', 'Label_3', 'Label_4', 'Label_5', 'Label_6']], axis=1))

Counter({0: 4, 1: 712, 2: 11, 3: 1})

### Примеры текстов из трейна

In [10]:
train['text'][0]

'. Об утверждении годового отчета ОАО « НЛМК » за 2007 год , годовой бухгалтерской отчетности , в том числе отчета о прибылях и об убытках ОАО « НЛМК » , а также распределение прибыли ( в том числе выплата дивидендов ) и убытков ОАО « НЛМК » по результатам 2007 финансового года Предлагаемые формулировки решений по рассматриваемому вопросу повестки дня : 1 . 1 . Утвердить годовой отчет ОАО « НЛМК » за 2007 год , годовую бухгалтерскую отчетность , отчет о прибылях и об убытках ОАО « НЛМК » , распределение прибыли и убытков ОАО « НЛМК » по результатам 2007 финансового года . 1 . 2 . Объявить дивиденды за 2007 год по размещенным обыкновенным акциям денежными средствами в размере 3 рубля на одну обыкновенную акцию . С учетом выплаченных промежуточных дивидендов за 1 полугодие 2007 года в размере 1 , 5 рубля на одну обыкновенную акцию , дополнительно выплатить 1 , 5 рубля на одну обыкновенную акцию . Выплату дивидендов произвести до 3 сентября 2008 года путем перечисления денежных средств : 

In [11]:
train['text'][1]

'2 . Об избрании членов Совета директоров ОАО « НЛМК » В соответствии с п . 4 ст . 66 Федерального закона РФ « Об акционерных обществах » по данному вопросу проводится кумулятивное голосование . Количественный состав Совета директоров в соответствии с Уставом Общества - девять человек . При подведении итогов голосования установлено следующее распределение голосов : Количество голосов для кумулятивного голосования , принадлежащее лицам , включенным в список лиц , имеющих право на участие в собрании Количество голосов для кумулятивного голосования , принадлежащее лицам , зарегистрированным для участия в собрании Наличие кворума Количество голосов , не принявшее участия в голосовании Количество голосов , принадлежащее лицам , распределившим голоса между кандидатами Количество голосов , принадлежащее лицам , указавшим вариант голосования " ПРОТИВ всех кандидатов " Количество голосов , принадлежащее лицам , указавшим вариант голосования " ВОЗДЕРЖАЛСЯ по всем кандидатам " Количество голосов 

In [12]:
train['text'][2]

'3 . Об избрании Президента ( Председателя Правления ) ОАО « НЛМК » Предлагаемая формулировка решения по рассматриваемому вопросу повестки дня : Избрать Президентом ( Председателем Правления ) ОАО « НЛМК » Лапшина Алексея Алексеевича . При подведении итогов голосования установлено следующее распределение голосов : Номер вопроса ( подпункта вопроса ) повестки дня Количество голосов , принадлежащее лицам , включённым в список лиц , имеющих право на участие в собрании Количество голосов , принадлежащее лицам , зарегистрированным для участия в собрании Наличие кворума Количество голосов , не принявшее участие в голосовании Количество голосов , принявшее участие в голосовании Количество голосов , принадлежащее лицам , указавшим вариант голосования " ЗА " Количество голосов , принадлежащее лицам , указавшим вариант голосования " ПРОТИВ " Количество голосов , принадлежащее лицам , указавшим вариант голосования " ВОЗДЕРЖАЛСЯ " Количество голосов в бюллетенях , признанных недействительными Прин

In [13]:
train['text'][3]

'4 . Об избрании членов Ревизионной комиссии ОАО « НЛМК » Количественный состав Ревизионной комиссии в соответствии с Уставом Общества - пять человек . Предлагаемые кандидаты для избрания членов Ревизионной комиссии Общества : 1 . Бизяева Надежда Андреевна ; 2 . Горбунова Татьяна Владимировна ; 3 . Кладиенко Людмила Владимировна ; 4 . Куликов Валерий Серафимович ; 5 . Овсянникова Лариса Михайловна ; 6 . Шипилова Галина Ивановна ; В соответствии с п . 6 ст . 85 Федерального закона РФ « Об акционерных обществах » в голосовании по выборам Ревизионной комиссии не принимают участия голоса по акциям , принадлежащим членам Правления и Совета директоров Общества . При подведении итогов голосования установлено следующее распределение голосов : Порядковый номер кандидата Количество голосов , принадлежащее лицам , включённым в список лиц , имеющих право на участие в собрании , и имеющим право голосовать по данному вопросу Количество голосов , принадлежащее лицам , зарегистрированным для участия в

In [14]:
train['text'][100]

'По итогам определения результатов распределения голосов по бюллетеням № 3 , вопросу № 11 повестки дня годового общего собрания акционеров ПАО « Татнефть » : « Утверждение « Положения о генеральном директоре публичного акционерного общества « Татнефть » имени В . Д . Шашина » в новой редакции » ПОДАНО ГОЛОСОВ : « ЗА » f 363 030 О / У « ПРОТИВ » 6 366 666 « ВОЗДЕРЖАЛСЯ » 36 600 Зб6 ~ Для принятия решения по вопросу № 11 бюллетеня № 3 повестки дня годового общего собрания акционеров необходимо набрать простое большинство голосов , то есть не менее 066 606 Q66 голосов акционеров - владельцев размещенных голосующих акций , принимающих участие в собрании . Таким образом , исходя из итогов голосования по вопросу № 11 бюллетеня № 3 повестки дня годового общего собрания акционеров , простое большинство голосов акционеров - владельцев размещенных голосующих акций , принимающих участие в собрании , набрано . Годовым общим собранием акционеров ПАО « Татнефть » принято решение : « Утвердить « Поло

In [15]:
train['text'][200]

'По вопросу № 4 « Об избрании членов Ревизионной комиссии Общества » информацию о кандидатах в Ревизионную комиссию Общества представила заместитель генерального директора по экономике и финансам Тихомирова О . В . Пунктом 24 . 1 статьи 24 Устава Общества состав Ревизионной комиссии Общества определен в количестве 5 ( Пяти ) человек . В соответствии со статьей 53 Федерального закона « Об акционерных обществах » и статьей 13 Устава Общества предложения с кандидатурами для выдвижения в Ревизионную комиссию ПАО « МРСК Центра и Приволжья » поступили в Совет директоров от акционера Общества - ПАО « Россети » . Совет директоров ПАО « МРСК Центра и Приволжья » на своем заседании 06 . 03 . 2017 в соответствии со статьей 53 Федерального закона « Об акционерных обществах » и статьей 13 Устава Общества , рассмотрел предложения акционера по выдвижению кандидатов в органы контроля Общества , принял решение включить в список кандидатур для голосования по выборам в Ревизионную комиссию предложенных к

In [16]:
train['text'][300]

"ВОПРОС № 3 . Об утверждении Устава ОАО « ДВЭУК » в новой редакции . Кворум по третьему вопросу повестки дня : Число голосов , которыми обладали лица , включенные в список лиц , имеющих право на участие во внеочередном Общем собрании акционеров по третьему вопросу повестки дня - 54 367 064 ( Пятьдесят четыре миллиона триста шестьдесят семь тысяч шестьдесят четыре ) голоса . Число голосов , приходившихся на голосующие акции Общества - 54 367 064 ( Пятьдесят четыре миллиона триста шестьдесят семь тысяч шестьдесят четыре ) голоса . Число голосов , которыми обладали лица , принявшие участие во внеочередном Общем собрании акционеров по третьему вопросу повестки дня - 54 367 064 ( Пятьдесят четыре миллиона триста шестьдесят семь тысяч шестьдесят четыре ) голоса . Кворум для принятия решения по третьему вопросу повестки дня внеочередного Общего собрания акционеров Общества имелся . СЛУШАЛИ : Начальника Службы корпоративного управления ОАО « ДВЭУК » Гепалова К . В . , который сообщил , что 27 

In [17]:
train['text'][400]

'. Избрание Исполнительного директора Общества . Описание : В связи с досрочным прекращением полномочий Исполнительного директора Общества Чермашенцева Евгения Николаевича ввиду расторжения заключенного с ним трудового договора , акционерам Общества предлагается принять решение об избрании Исполнительным директором Общества Вермишяна Геворка Арутюновича . Решение : Избрать Вермишяна Геворка Арутюновича Исполнительным директором Общества сроком до даты проведения годового Общего собрания акционеров Общества в 2020 году ( включительно ) . Итоги голосования по первому вопросу Повестки дня внеочередного Общего собрания акционеров : Число голосов , которыми обладали лица , включенные в список лиц , имевших право на участие в общем собрании , по данному вопросу повестки дня общего собрания 620 000 000 Число голосов , приходившихся на голосующие акции общества по данному вопросу повестки дня общего собрания , определенное с учетом положений пункта 4 . 20 Положения 620 000 000 Число голосов , 

In [18]:
train['text'][500]

'Результаты кумулятивного голосования по вопросу № 2 повестки дня : Число кумулятивных голосов , которыми обладали лица , включенные в список лиц , имевших право на участие в общем собрании , по вопросу № 2 повестки дня - 284 984 328 . Число кумулятивных голосов , приходившихся на голосующие акции , определенное с учетом положений пункта 4 . 20 Приказа ФСФР России от 02 . 02 . 2012 № 12 - 6 / пз - н по вопросу № 2 повестки дня : 284 984 328 . Число кумулятивных голосов , которыми обладали лица , принявшие участие в общем собрании , по вопросу № 2 повестки дня : 239 298 576 . Кворум для голосования по вопросу повестки дня № 2 имеется . Число голосов , отданных за каждый вариант голосования по вопросу № 2 повестки дня : « за » - распределение голосов по кандидатам : 1 . Гончарова Людмила Антоновна - 19 793 253 ; 2 . Долгополов Александр Витальевич - 19 796 454 ; 3 . Зацепин Николай Степанович - 19 530 995 ; 4 . Куимов Игорь Евгеньевич - 19 934 031 ; 5 . Лапушкин Олег Иванович - 19 933 41

In [19]:
train['text'][600]

'регистрации и подсчета голосов по вопросу 1 : О досрочном прекращении полномочий членов Совета Директоров Банка " Возрождение " ( ПАО ) . Число голосов , которыми обладают лица , включенные в список лиц , имеющих право на участие в общем собрании по данному вопросу повестки дня общего собрания - 23 748 694 . Число голосов , приходившихся на голосующие акции общества по данному вопросу повестки дня общего собрания , определенное с учетом положений пункта 4 . 20 Приказа ФСФР России № 12 - 6 / лз - н от 02 . 02 . 2012г . " Об Утверждении Положения о дополнительных требованиях к порядку подготовки , созыва и проведения общего собрания акционеров " , предписания Банка России № 015 - 55 / 1616 от 06 . 03 . 2018 г . и требований ст . 61 Федерального закона « О Центральном Банке России » № 86 - ФЗ от 27 . 06 . 2002 г . - 13 601 109 . Число голосов , которыми обладают лица , принявшие участие в общем собрании по данному вопросу повестки дня общего собрания - 12105 566 , что составляет 89 . 00 

In [20]:
train['text'][700]

'По вопросу 4 повестки дня : Проект решения : Избрать Совет директоров Общества в следующем составе : 1 . Анипкин Максим Николаевич ; 2 . Антонюк Алексей Владимирович ; 3 . Быстрых Евгений Александрович ; 4 . Вейалайнен Яркко Армас ; 5 . Есиков Александр Юрьевич ; 6 . Каплун Павел Сергеевич ; 7 . Копонен Харри Ээрик ; 8 . Ушков Александр Анатольевич ; 9 . Чумаченко Наталья Викторовна . Итоги голосования по четвертому вопросу повестки дня годового Общего собрания акционеров : Число голосов , которыми обладали лица , включенные в список лиц , имевших право на участие в общем собрании , по данному вопросу повестки дня общего собрания 5 580 000 000 Число голосов , приходившихся на голосующие акции общества по данному вопросу повестки дня общего собрания , определенное с учетом положений пункта 4 . 20 Положения 5 580 000 000 Число голосов , которыми обладали лица , принявшие участие в общем собрании , по данному вопросу повестки дня общего собрания 4 574 378 038 КВОРУМ по данному вопросу им

### Выглядят достаточно однородно: голосования и принятия каких-то решений

### Проверим, что в тесте лежит то же самое

In [352]:
test

,text
0,ВОПРОС № 6 . О выплате вознаграждения за работ...
1,ВОПРОС № 7 . Избрание членов Совета директоров...
2,ВОПРОС № 8 . Избрание членов Ревизионной комис...
3,ВОПРОС № 9 . Утверждение аудитора Общества . К...
4,ВОПРОС № 10 . Об уменьшении уставного капитала...
5,ВОПРОС № И . Об утверждении Устава Общества в ...
6,ВОПРОС № 12 . Об утверждении Положении об инфо...
7,регистрации и подсчета голосов по вопросу 1 : ...
8,Результаты регистрации и подсчета голосов по в...
9,№ 1 : Утверждение годового отчета Общества . К...


In [353]:
test['text'][0]

'ВОПРОС № 6 . О выплате вознаграждения за работу в составе Ревизионной комиссии членам Ревизионной комиссии - негосударственным служащим в размере , усi ановленном внутренними документами Общества . Кворум по шестому вопросу повестки дня : Число голосов , которыми обладали лица , включенные в список лиц . имеющих право на участие в годовом Общем собрании акционеров по шестому вопросу повестки дня 60 200 643 ( Шестьдесят миллионов двести тысяч шестьсот сорок три ) голоса . Число голосов , приходившихся на голосующие акции Общества , определенное в соответствии с п . 4 . 20 Положения о дополнительных требованиях к порядку подготовки , созыва и проведения общего собрания акционеров . 60 200 643 ( Шестьдесят миллионов двести тысяч шестьсот сорок три ) голоса . Число голосов , которыми обладали лица , принявшие участие в годовом Общем собрании акционеров по шестому вопросу повестки дня 60 200 493 ( Шестьдесят миллионов двести тысяч четыреста девяносто три ) голоса . Кворум для принятия реше

In [354]:
test['text'][50]

'вопросу повестки дня : Число голосов , которыми обладали лица , включенные в список лиц , имеющих право на участие в Собрании , по данному вопросу повестки дня : 837 718 660 штук . Число голосов , которыми обладали лица , принявшие участие в голосовании по данному вопросу повестки дня : 765 062 078 штук . Кворум имелся . Итоги голосования : Число голосов , принадлежащих лицам , участвовавшим в голосовании по вопросу повестки дня : 765 062 078 . Число голосов « За » : 717 503 494 ; « Против » : 100 ; « Воздержался » : 56 704 ; число голосов , представленных бюллетенями , признанными недействительными : 697 591 . Голоса , не распределенные при голосовании акционером , голосующим в соответствии с указанием владельцев депозитарных ценных бумаг : 46 804 189 . По вопросу повестки дня большинством голосов принято решение : Выплатить ( объявить ) дивиденды по результатам 9 месяцев 2012 года в размере 3 рублей 18 копеек на одну обыкновенную именную акцию . Форма выплаты дивидендов : безналичны

### Переведем весь текст в нижний регистр

In [21]:
train['text'] = [x.lower() for x in train['text']]
test['text'] = [x.lower() for x in test['text']]

### Частотность токенов для одного из примеров

In [22]:
Counter(train['text'][0].split(' ')).most_common()

[(',', 54),
 ('в', 37),
 ('.', 28),
 ('«', 28),
 ('»', 28),
 ('оао', 25),
 ('нлмк', 21),
 ('по', 20),
 ('1', 19),
 ('-', 16),
 ('%', 16),
 ('5', 12),
 ('2007', 10),
 ('(', 10),
 (')', 10),
 ('порядке', 10),
 ('на', 9),
 ('голосов', 9),
 ('за', 8),
 ('и', 8),
 ('количество', 8),
 ('года', 7),
 ('для', 7),
 ('лиц', 7),
 ('3', 6),
 ('рубля', 6),
 ('одну', 6),
 ('обыкновенную', 6),
 ('акцию', 6),
 ('акционеров', 6),
 ('или', 6),
 ('безналичном', 6),
 ('платежным', 6),
 ('реквизитам', 6),
 ('"', 6),
 ('0', 6),
 ('об', 5),
 ('год', 5),
 ('дивидендов', 5),
 (':', 5),
 ('голосования', 5),
 ('принадлежащее', 5),
 ('лицам', 5),
 ('распределение', 4),
 ('результатам', 4),
 ('отчет', 4),
 ('2', 4),
 ('размере', 4),
 ('до', 4),
 ('денежных', 4),
 ('средств', 4),
 ('платежа', 4),
 ('указанным', 4),
 ('анкете', 4),
 ('зарегистрированного', 4),
 ('лица', 4),
 ('у', 4),
 ('регистратора', 4),
 (';', 4),
 ('физических', 4),
 ('наличном', 4),
 ('кассе', 4),
 ('липецккомбанк', 4),
 ('да', 4),
 ('019', 4),


### Частотность токенов для всех примеров их трейна

In [23]:
all_text = ''.join([train['text'][i] for i in range(train.shape[0])])


In [24]:
Counter(all_text.split(' ')).most_common()

[('.', 17746),
 (',', 16421),
 ('в', 8106),
 ('по', 5993),
 ('«', 5705),
 ('»', 5575),
 (':', 4788),
 (')', 4636),
 ('голосов', 4636),
 ('(', 4521),
 ('%', 3452),
 ('вопросу', 3440),
 ('-', 3329),
 ('дня', 3254),
 ('и', 3232),
 ('повестки', 3199),
 ('число', 3044),
 ('на', 2883),
 ('0', 2709),
 ('общества', 2633),
 ('1', 2590),
 ('"', 2259),
 ('за', 2095),
 ('собрания', 2086),
 ('2', 2058),
 ('с', 2034),
 ('голосования', 2005),
 ('общего', 1790),
 ('акционеров', 1748),
 ('участие', 1645),
 ('данному', 1633),
 ('№', 1602),
 ('оао', 1584),
 ('директоров', 1563),
 ('5', 1548),
 ('собрании', 1546),
 (';', 1508),
 ('4', 1496),
 ('о', 1427),
 ('лица', 1354),
 ('3', 1323),
 ('от', 1202),
 ('/', 1198),
 ('или', 1166),
 ('совета', 1100),
 ('не', 1098),
 ('которыми', 1098),
 ('пао', 1079),
 ('общем', 1060),
 ('обладали', 1045),
 ('решение', 970),
 ('000', 953),
 ('6', 901),
 ('решения', 882),
 ('года', 872),
 ('лиц', 870),
 ('об', 849),
 ('право', 845),
 ('п', 812),
 ('100', 800),
 ('против', 77

### Очень много терминов в кавычках, посмотрим на их частотность

In [25]:
in_quotes = re.findall('(« [a-zA-Zа-яА-Я0-9]+ »|[“””"] [a-zA-Zа-яА-Я0-9]+ [“””"])', all_text)
in_quotes = [x[2:-2] for x in in_quotes]
Counter(in_quotes).most_common()

[('за', 682),
 ('против', 650),
 ('двэук', 642),
 ('нлмк', 638),
 ('воздержался', 636),
 ('возрождение', 591),
 ('новатэк', 211),
 ('недействительные', 134),
 ('лукойл', 65),
 ('трансконтейнер', 58),
 ('татнефть', 55),
 ('роснефть', 52),
 ('мегафон', 51),
 ('мечел', 45),
 ('липецккомбанк', 39),
 ('башэнергоактив', 33),
 ('северсталь', 30),
 ('якутуголь', 24),
 ('газпромбанк', 23),
 ('чтпз', 23),
 ('пнтз', 22),
 ('чмк', 21),
 ('россети', 16),
 ('зарплатный', 15),
 ('кпмг', 15),
 ('уралкуз', 13),
 ('интернет', 12),
 ('онс', 12),
 ('экоресурс', 12),
 ('башкирэнерго', 10),
 ('банк', 8),
 ('энергострой', 8),
 ('эмк', 8),
 ('альфастрахование', 8),
 ('директориум', 6),
 ('feemc', 6),
 ('рэа', 6),
 ('дрск', 6),
 ('русгидро', 6),
 ('бмк', 6),
 ('положение', 5),
 ('россия', 5),
 ('банки', 4),
 ('заемщик', 4),
 ('росбанк', 4),
 ('профитек', 4),
 ('заказчик', 4),
 ('городская', 4),
 ('пеледуй', 4),
 ('компания', 4),
 ('кредитор', 4),
 ('транскредитбанк', 3),
 ('лидер', 3),
 ('мкс', 3),
 ('финэскор

### Сделаем фичи на основе структуры текста: количество и число уникальных названий в кавычках, отдельно факта голосований, наличие списков с нумерацией, количество предложений, слов, чисел, знаков двоеточия и процента

In [26]:
def text_structure_features(text):
    
    quotes_doc = re.findall('(« [a-zA-Zа-яА-Я0-9]+ »|[“””"] [a-zA-Zа-яА-Я0-9]+ [“””"])', text)
    quotes_doc = [x[2:-2] for x in quotes_doc]
    quotes_voting = len([x for x in quotes_doc if x in ['за', 'против', 'воздержался']])
    quotes_doc = [x for x in quotes_doc if not(x in ['за', 'против', 'воздержался'])]
    quotes_count = len(quotes_doc)
    quotes_unq = len(np.unique(quotes_doc))
    
    brackets = re.findall('\(.*?\)', text)
    brackets_count = len(brackets)
    brackets_unq = len(np.unique(brackets))
    
    listings = len(re.findall('(\d+ \. [a-zA-Zа-яА-Я]+)', text))            
    
    sentence_count = len(re.findall('([a-zA-Zа-яА-Я]{3,} \.|[0-9]{3} \.|[a-zA-Zа-яА-Я0-9]+ [“””"»\)] \.)', text))
    sentence_count = sentence_count if text[-1:]=='.' else sentence_count+1
    sentence_count = np.max([1,sentence_count])
    
    words = len(re.findall('([a-zA-Zа-яА-Я]+)', text))
                
    numbers = len(re.findall('([0-9]+[0-9 ]*)', text))            
    
    colons = len(re.findall(':', text))
                
    percents = len(re.findall('%', text))            
    
    return([quotes_voting, quotes_count, quotes_unq, brackets_count, brackets_unq, listings, sentence_count, words, numbers, colons, percents])
    

In [27]:
train = pd.concat([train, pd.DataFrame([text_structure_features(train['text'][i]) for i in range(train.shape[0])],
                                    columns=['quotes_voting', 'quotes_count', 'quotes_unq', 'brackets_count', 'brackets_unq', 'listings', 'sentence_count',
                                             'words', 'numbers', 'colons', 'percents'])], axis=1)

### Сделаем частотные фичи, нормируя на количество предложений в тексте

In [28]:
train['words_freq'] = train['words']/train['sentence_count']
train['numbers_freq'] = train['numbers']/train['sentence_count']
train['quotes_freq'] = train['quotes_count']/train['sentence_count']
train['brackets_freq'] = train['brackets_count']/train['sentence_count']

###  Судя по количеству лейблов в трейне, можно попробовать построить и оценить качество модели для 4, 5, 6 и 3-го лейбла, хотя для последних двух результаты будут конечно не очень устойчивы

In [103]:
np.sum(train[labels])

Label_1      6
Label_2      8
Label_3     14
Label_4    601
Label_5     84
Label_6     24
dtype: int64

### Разобьем трейн на сеты для обучения и валидации

In [163]:
test_index = random.sample(list(train.index), int(train.shape[0]/3))
train_index = [x for x in train.index if not(x in test_index)]
shuffle(train_index)

### И построим по отдельной модели для каждого из этих четырех лейблов. Для этого обучим xgboost без особого тюнинга параметров (так как у него даже в этом случае перформанс будет близок к наилучшему) и посмотрим на качество предсказаний для каждого из лейблов

In [184]:
cols=['quotes_voting', 'quotes_count', 'quotes_unq',
       'brackets_count', 'brackets_unq', 'listings', 'sentence_count', 'words',
       'numbers', 'colons', 'percents', 'words_freq', 'numbers_freq',
       'quotes_freq', 'brackets_freq']
labels=['Label_1', 'Label_2', 'Label_3', 'Label_4', 'Label_5', 'Label_6']

In [165]:
n_parallel = 7

param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'auc'
param['nthread'] = n_parallel

param['eta'] = 0.1
param['max_depth'] = 4
param['min_child_weight'] = 1
param['subsample'] = 1
param['colsample_bytree'] = 1
param['colsample_bylevel'] = 1
param['lambda'] = 1
param['alpha'] = 0
param['seed'] = 0
param['gamma'] = 0

numround = 100



In [166]:
def run_xgb(label, cols):
    dtrain = xgb.DMatrix(train.ix[train_index][cols], label=train.ix[train_index][label])
    dtest = xgb.DMatrix(train.ix[test_index][cols], label=train.ix[test_index][label])
    evallist = [(dtest,'test'), (dtrain,'train')]
    
    model = xgb.train(param, dtrain, numround, evallist)

In [322]:
run_xgb('Label_4', cols)

[0]	test-auc:0.79018	train-auc:0.809732
[1]	test-auc:0.781915	train-auc:0.813322
[2]	test-auc:0.781089	train-auc:0.820468
[3]	test-auc:0.783617	train-auc:0.822128
[4]	test-auc:0.789645	train-auc:0.85576
[5]	test-auc:0.807632	train-auc:0.874262
[6]	test-auc:0.80598	train-auc:0.876929
[7]	test-auc:0.837725	train-auc:0.88701
[8]	test-auc:0.845017	train-auc:0.88966
[9]	test-auc:0.838648	train-auc:0.902426
[10]	test-auc:0.83894	train-auc:0.90417
[11]	test-auc:0.853719	train-auc:0.917858
[12]	test-auc:0.863685	train-auc:0.922487
[13]	test-auc:0.872193	train-auc:0.937349
[14]	test-auc:0.886534	train-auc:0.941945
[15]	test-auc:0.886923	train-auc:0.948269
[16]	test-auc:0.891638	train-auc:0.951339
[17]	test-auc:0.895771	train-auc:0.955431
[18]	test-auc:0.904278	train-auc:0.961638
[19]	test-auc:0.907681	train-auc:0.966636
[20]	test-auc:0.90525	train-auc:0.969656
[21]	test-auc:0.907487	train-auc:0.97083
[22]	test-auc:0.903889	train-auc:0.973111
[23]	test-auc:0.901847	train-auc:0.973648
[24]	test-a

In [323]:
run_xgb('Label_5', cols)

[0]	test-auc:0.959345	train-auc:0.952007
[1]	test-auc:0.977549	train-auc:0.968702
[2]	test-auc:0.978627	train-auc:0.967371
[3]	test-auc:0.970537	train-auc:0.966039
[4]	test-auc:0.982268	train-auc:0.987253
[5]	test-auc:0.983819	train-auc:0.987253
[6]	test-auc:0.98483	train-auc:0.990083
[7]	test-auc:0.983482	train-auc:0.988846
[8]	test-auc:0.984628	train-auc:0.991272
[9]	test-auc:0.986179	train-auc:0.990035
[10]	test-auc:0.986044	train-auc:0.990321
[11]	test-auc:0.986179	train-auc:0.993793
[12]	test-auc:0.987392	train-auc:0.99434
[13]	test-auc:0.987931	train-auc:0.994815
[14]	test-auc:0.989887	train-auc:0.995529
[15]	test-auc:0.990561	train-auc:0.996005
[16]	test-auc:0.990426	train-auc:0.99629
[17]	test-auc:0.991235	train-auc:0.996575
[18]	test-auc:0.99164	train-auc:0.996813
[19]	test-auc:0.992044	train-auc:0.997146
[20]	test-auc:0.991909	train-auc:0.997598
[21]	test-auc:0.99164	train-auc:0.997646
[22]	test-auc:0.992179	train-auc:0.997836
[23]	test-auc:0.992044	train-auc:0.998216
[24]	te

In [324]:
run_xgb('Label_6', cols)

[0]	test-auc:0.557458	train-auc:0.635632
[1]	test-auc:0.557458	train-auc:0.635632
[2]	test-auc:0.557654	train-auc:0.635713
[3]	test-auc:0.557654	train-auc:0.635713
[4]	test-auc:0.559622	train-auc:0.636933
[5]	test-auc:0.560016	train-auc:0.637095
[6]	test-auc:0.560212	train-auc:0.637421
[7]	test-auc:0.561	train-auc:0.637583
[8]	test-auc:0.560212	train-auc:0.637421
[9]	test-auc:0.647383	train-auc:0.853147
[10]	test-auc:0.644235	train-auc:0.856074
[11]	test-auc:0.651122	train-auc:0.869572
[12]	test-auc:0.658205	train-auc:0.875915
[13]	test-auc:0.66096	train-auc:0.880631
[14]	test-auc:0.662928	train-auc:0.881688
[15]	test-auc:0.665289	train-auc:0.88429
[16]	test-auc:0.66647	train-auc:0.890063
[17]	test-auc:0.670799	train-auc:0.892015
[18]	test-auc:0.669225	train-auc:0.891039
[19]	test-auc:0.672176	train-auc:0.892665
[20]	test-auc:0.672176	train-auc:0.893479
[21]	test-auc:0.672963	train-auc:0.893479
[22]	test-auc:0.674144	train-auc:0.893479
[23]	test-auc:0.673947	train-auc:0.894454
[24]	tes

In [325]:
run_xgb('Label_3', cols)

[0]	test-auc:0.495763	train-auc:0.559362
[1]	test-auc:0.495763	train-auc:0.559362
[2]	test-auc:0.495763	train-auc:0.559362
[3]	test-auc:0.812147	train-auc:0.857087
[4]	test-auc:0.812147	train-auc:0.857087
[5]	test-auc:0.812147	train-auc:0.857087
[6]	test-auc:0.812147	train-auc:0.857087
[7]	test-auc:0.811441	train-auc:0.855779
[8]	test-auc:0.893715	train-auc:0.99307
[9]	test-auc:0.893008	train-auc:0.991501
[10]	test-auc:0.893715	train-auc:0.994116
[11]	test-auc:0.887006	train-auc:0.996208
[12]	test-auc:0.887006	train-auc:0.996208
[13]	test-auc:0.887006	train-auc:0.996862
[14]	test-auc:0.887006	train-auc:0.997908
[15]	test-auc:0.887006	train-auc:0.997908
[16]	test-auc:0.887006	train-auc:0.998431
[17]	test-auc:0.887006	train-auc:0.998431
[18]	test-auc:0.881356	train-auc:0.998431
[19]	test-auc:0.881356	train-auc:0.998431
[20]	test-auc:0.88065	train-auc:0.998692
[21]	test-auc:0.88065	train-auc:0.998692
[22]	test-auc:0.88065	train-auc:0.998692
[23]	test-auc:0.88065	train-auc:0.998692
[24]	te

### Ниже полученные метрики AUC 

In [ ]:
########### text_structure ###
### Label_4     0.912
### Label_5     0.991
### Label_6     0.742
### Label_3     0.893

### Посмотрим на частоту использования фичей в моделях для разных лейблов

In [141]:
dtrain = xgb.DMatrix(train.ix[train_index][cols], label=train.ix[train_index]['Label_4'])
dtest = xgb.DMatrix(train.ix[test_index][cols], label=train.ix[test_index]['Label_4'])
evallist = [(dtest,'test'), (dtrain,'train')]
    
model = xgb.train(param,dtrain,numround,evallist, verbose_eval=False)

importance = model.get_fscore()
importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)
importance

[('listings', 96),
 ('numbers_freq', 91),
 ('words', 90),
 ('numbers', 84),
 ('quotes_freq', 80),
 ('words_freq', 64),
 ('colons', 60),
 ('percents', 59),
 ('brackets_freq', 55),
 ('brackets_count', 53),
 ('brackets_unq', 32),
 ('quotes_count', 30),
 ('quotes_voting', 30),
 ('quotes_unq', 6),
 ('sentence_count', 3)]

In [142]:
dtrain = xgb.DMatrix(train.ix[train_index][cols], label=train.ix[train_index]['Label_6'])
dtest = xgb.DMatrix(train.ix[test_index][cols], label=train.ix[test_index]['Label_6'])
evallist = [(dtest,'test'), (dtrain,'train')]
    
model = xgb.train(param, dtrain, numround, evallist, verbose_eval=False)

importance = model.get_fscore()
importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)
importance

[('words', 97),
 ('words_freq', 66),
 ('numbers', 50),
 ('quotes_count', 48),
 ('numbers_freq', 42),
 ('quotes_freq', 42),
 ('brackets_freq', 40),
 ('colons', 24),
 ('percents', 20),
 ('quotes_voting', 19),
 ('listings', 17),
 ('brackets_count', 12),
 ('sentence_count', 12),
 ('brackets_unq', 8)]

###  Испробуем фичи на основе встречаемости слов (tf-idf) на очищенном лемматизированном тексте

In [356]:
### Уберем стоп-слова

In [174]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/enke/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [182]:
nltk.corpus.stopwords.words('russian')

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

### Лемматизация русского текста есть в пакете stanfordnlp

In [205]:
import stanfordnlp
from spacy_stanfordnlp import StanfordNLPLanguage

stanfordnlp.download("ru")

snlp = stanfordnlp.Pipeline(lang="ru")
nlp = StanfordNLPLanguage(snlp)

Using the default treebank "ru_syntagrus" for language "ru".
Would you like to download the models for: ru_syntagrus now? (Y/n)
Y

Default download directory: /Users/enke/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /Users/enke/stanfordnlp_resources/ru_syntagrus_models.zip


100%|██████████| 236M/236M [05:25<00:00, 777KB/s]  



Download complete.  Models saved to: /Users/enke/stanfordnlp_resources/ru_syntagrus_models.zip
Extracting models file for: ru_syntagrus
Cleaning up...Done.
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/enke/stanfordnlp_resources/ru_syntagrus_models/ru_syntagrus_tokenizer.pt', 'lang': 'ru', 'shorthand': 'ru_syntagrus', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/enke/stanfordnlp_resources/ru_syntagrus_models/ru_syntagrus_tagger.pt', 'pretrain_path': '/Users/enke/stanfordnlp_resources/ru_syntagrus_models/ru_syntagrus.pretrain.pt', 'lang': 'ru', 'shorthand': 'ru_syntagrus', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/enke/stanfordnlp_resources/ru_syntagrus_models/ru_syntagrus_lemmatizer.pt', 'lang': 'ru', 'shorthand': 'ru_syntagrus', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmat

In [211]:
def normalize_words(text):
    tokens = re.findall('([a-zA-Zа-яА-Я]{3,})', text)
    tokens = [i for i in tokens if not i in nltk.corpus.stopwords.words('russian')]
    tokens = ' '.join(tokens)
    tokens = ' '.join([token.lemma_ for token in nlp(tokens)])
    return(tokens)
    

### Работает медленно, для больших датасетов или ответов в реальном времени может не подойти

In [213]:
%%time
normalize_words(train['text'][700])

CPU times: user 4.14 s, sys: 354 ms, total: 4.49 s
Wall time: 2.18 s


'вопрос повестка день проект решение избрать совет директор общество следующий состав анипкин максим николаевич антонюк алекс владимирович быстрый евгений александрович вейалайний яркко армас есик александр юрьевич каплун павел сергеевич копонный харри ээрик ушк александр анатольевич чумаченко наталья викторовна итог голосование четвертый вопрос повестка день годовой общий собрание акционер число голос который обладать лицо включить список лицо иметь право участие общий собрание данный вопрос повестка день общий собрание число голос приходиться голосовать акция общество данный вопрос повестка день общий собрание определить учет положение пункт положение число голос который обладать лицо принять участие общий собрание данный вопрос повестка день общий собрание кворум данный вопрос иметься кандидат число голос отдать каждый вариант голосование распределение голос кандидат быстрый евгений александрович чумаченко наталья викторовна есик александр юрьевич анипкин максим николаевич каплун па

In [223]:
train['text_norm'] = [normalize_words(train['text'][i]) for i in range(train.shape[0])]

### Сохраним, чтобы не пришлось пересчитывать, очень уж долго

In [224]:
#train.to_csv('train_norm.csv', index=False, sep='\t')

In [313]:
#train = pd.read_csv('train_norm.csv', sep='\t') 

### Сделаем tf-idf для 1- и 2-грам, чтобы уловить часто встречающие словосочетания, и возьмем из них 200 топовых. Строить tf-idf будем только на сете для обучения, чтобы не использовать данные из валидации, которые мы не должны видеть

In [315]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [316]:
tfidf_feautures = 200

vectorizer = TfidfVectorizer(max_features=tfidf_feautures, ngram_range=(1, 2))

tfidf_cols = ['tfidf_'+str(i) for i in range(tfidf_feautures)]

vectorizer = vectorizer.fit(train.ix[train_index]['text_norm'])
tfidf = vectorizer.transform(train['text_norm'])
tfidf = pd.DataFrame(tfidf.todense(), columns=tfidf_cols)

In [317]:
train = pd.concat([train,tfidf], axis=1)

In [319]:
run_xgb('Label_4', tfidf_cols)

[0]	test-auc:0.899562	train-auc:0.961202
[1]	test-auc:0.899562	train-auc:0.961202
[2]	test-auc:0.89859	train-auc:0.975309
[3]	test-auc:0.970588	train-auc:0.996679
[4]	test-auc:0.980846	train-auc:0.997132
[5]	test-auc:0.979971	train-auc:0.996427
[6]	test-auc:0.978123	train-auc:0.996226
[7]	test-auc:0.979631	train-auc:0.999061
[8]	test-auc:0.979242	train-auc:0.998222
[9]	test-auc:0.979874	train-auc:0.999396
[10]	test-auc:0.984541	train-auc:0.999262
[11]	test-auc:0.987312	train-auc:0.999564
[12]	test-auc:0.989645	train-auc:0.999665
[13]	test-auc:0.989548	train-auc:0.999665
[14]	test-auc:0.990034	train-auc:0.999665
[15]	test-auc:0.990666	train-auc:0.999799
[16]	test-auc:0.991055	train-auc:0.999866
[17]	test-auc:0.991249	train-auc:1
[18]	test-auc:0.991249	train-auc:1
[19]	test-auc:0.990083	train-auc:1
[20]	test-auc:0.991249	train-auc:1
[21]	test-auc:0.99193	train-auc:1
[22]	test-auc:0.992124	train-auc:1
[23]	test-auc:0.992416	train-auc:1
[24]	test-auc:0.992319	train-auc:1
[25]	test-auc:0.99

In [326]:
run_xgb('Label_5', tfidf_cols)

[0]	test-auc:0.983414	train-auc:0.999358
[1]	test-auc:0.983414	train-auc:0.999358
[2]	test-auc:0.98483	train-auc:0.999643
[3]	test-auc:0.984156	train-auc:0.999738
[4]	test-auc:0.984695	train-auc:0.999834
[5]	test-auc:0.984628	train-auc:0.999929
[6]	test-auc:0.983482	train-auc:0.999976
[7]	test-auc:0.983482	train-auc:0.999976
[8]	test-auc:0.983212	train-auc:1
[9]	test-auc:0.983077	train-auc:1
[10]	test-auc:0.983077	train-auc:1
[11]	test-auc:0.98301	train-auc:1
[12]	test-auc:0.98301	train-auc:1
[13]	test-auc:0.98301	train-auc:1
[14]	test-auc:0.98301	train-auc:1
[15]	test-auc:0.98301	train-auc:1
[16]	test-auc:0.98301	train-auc:1
[17]	test-auc:0.98301	train-auc:1
[18]	test-auc:0.98301	train-auc:1
[19]	test-auc:0.982942	train-auc:1
[20]	test-auc:0.982605	train-auc:1
[21]	test-auc:0.982605	train-auc:1
[22]	test-auc:0.982605	train-auc:1
[23]	test-auc:0.982403	train-auc:1
[24]	test-auc:0.982403	train-auc:1
[25]	test-auc:0.982268	train-auc:1
[26]	test-auc:0.981931	train-auc:1
[27]	test-auc:0.98

In [327]:
run_xgb('Label_6', tfidf_cols)

[0]	test-auc:0.708186	train-auc:0.875183
[1]	test-auc:0.820543	train-auc:0.947715
[2]	test-auc:0.810508	train-auc:0.943731
[3]	test-auc:0.896891	train-auc:0.991869
[4]	test-auc:0.902794	train-auc:0.993983
[5]	test-auc:0.904959	train-auc:0.997886
[6]	test-auc:0.911649	train-auc:0.999024
[7]	test-auc:0.909681	train-auc:0.998211
[8]	test-auc:0.906139	train-auc:0.999187
[9]	test-auc:0.906139	train-auc:0.999349
[10]	test-auc:0.903385	train-auc:0.999349
[11]	test-auc:0.905352	train-auc:0.999349
[12]	test-auc:0.896497	train-auc:0.999349
[13]	test-auc:0.942739	train-auc:0.999512
[14]	test-auc:0.956316	train-auc:0.999837
[15]	test-auc:0.947658	train-auc:0.999837
[16]	test-auc:0.955529	train-auc:1
[17]	test-auc:0.950413	train-auc:1
[18]	test-auc:0.951594	train-auc:1
[19]	test-auc:0.949626	train-auc:1
[20]	test-auc:0.959071	train-auc:1
[21]	test-auc:0.962613	train-auc:1
[22]	test-auc:0.965368	train-auc:1
[23]	test-auc:0.962613	train-auc:1
[24]	test-auc:0.967729	train-auc:1
[25]	test-auc:0.968516	

In [328]:
run_xgb('Label_3', tfidf_cols)

[0]	test-auc:0.824859	train-auc:0.99477
[1]	test-auc:0.824859	train-auc:0.99477
[2]	test-auc:0.824859	train-auc:0.99477
[3]	test-auc:0.824859	train-auc:0.99477
[4]	test-auc:0.824859	train-auc:0.99477
[5]	test-auc:0.824506	train-auc:0.996862
[6]	test-auc:0.824506	train-auc:0.996862
[7]	test-auc:0.822034	train-auc:0.998954
[8]	test-auc:0.822034	train-auc:0.998954
[9]	test-auc:0.821328	train-auc:0.999739
[10]	test-auc:0.821328	train-auc:0.999739
[11]	test-auc:0.820621	train-auc:0.999739
[12]	test-auc:0.887359	train-auc:0.999739
[13]	test-auc:0.887359	train-auc:0.999739
[14]	test-auc:0.898658	train-auc:0.999739
[15]	test-auc:0.901483	train-auc:0.999739
[16]	test-auc:0.901483	train-auc:0.999739
[17]	test-auc:0.901483	train-auc:0.999739
[18]	test-auc:0.901483	train-auc:0.999739
[19]	test-auc:0.900777	train-auc:0.999739
[20]	test-auc:0.900777	train-auc:0.999739
[21]	test-auc:0.900777	train-auc:0.999739
[22]	test-auc:0.900777	train-auc:0.999739
[23]	test-auc:0.902189	train-auc:0.999215
[24]	te

### Стало лучше, для трех лейблов совсем хорошо, но в последнем возможно шум

In [ ]:
########### text_structure ### tfidf ###
### Label_4     0.912          0.996
### Label_5     0.991          0.993
### Label_6     0.742          0.982
### Label_3     0.893          0.908

### Теперь время попробовать предобученные модели. Хотела использовать Flair, но для русского там мало что оказалось в наличии. А вот в DeepPavlov с ним получше

### Попробуем извлечь NER

In [35]:
from deeppavlov import configs, build_model

In [ ]:
# python -m deeppavlov install ner_rus_bert

### Берем топовую модель ner_rus_bert

In [39]:
ner_model = build_model(configs.ner.ner_rus_bert, download=True)

2019-09-16 23:32:40.794 INFO in 'deeppavlov.core.data.utils'['utils'] at line 64: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz to /Users/enke/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz


2019-09-16 23:32:40,794 Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz to /Users/enke/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz


100%|██████████| 666M/666M [01:34<00:00, 7.09MB/s] 
2019-09-16 23:34:14.807 INFO in 'deeppavlov.core.data.utils'['utils'] at line 216: Extracting /Users/enke/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz archive into /Users/enke/.deeppavlov/downloads/bert_models


2019-09-16 23:34:14,807 Extracting /Users/enke/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz archive into /Users/enke/.deeppavlov/downloads/bert_models


2019-09-16 23:34:32.165 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz download because of matching hashes


...

2019-09-16 23:34:32,723 [loading vocabulary from /Users/enke/.deeppavlov/models/ner_rus_bert/tag.dict]
2019-09-16 23:34:32,743 From //anaconda/lib/python3.6/site-packages/deeppavlov/core/models/tf_model.py:38: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-09-16 23:34:32,747 From //anaconda/lib/python3.6/site-packages/deeppavlov/core/models/tf_model.py:223: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

2019-09-16 23:34:32,747 From //anaconda/lib/python3.6/site-packages/deeppavlov/core/models/tf_model.py:223: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

2019-09-16 23:34:32,878 From //anaconda/lib/python3.6/site-packages/deeppavlov/core/models/tf_model.py:194: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

2019-09-16 23:34:32,912 From //anaconda/lib/python3.6/site-packages/deeppavlov/models/bert/bert_ner.py:126: Th

Using TensorFlow backend.


2019-09-16 23:34:32,998 From //anaconda/lib/python3.6/site-packages/bert_dp/modeling.py:178: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

2019-09-16 23:34:34,675 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

2019-09-16 23:34:34,709 From //anaconda/lib/python3.6/site-packages/bert_dp/modeling.py:366: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-09-16 23:34:34,761 From //anaconda/lib/python3.6/site-packages/bert_dp/modeling.py:680: dens

2019-09-16 23:35:13.464 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /Users/enke/.deeppavlov/models/ner_rus_bert/model]


2019-09-16 23:35:13,464 [loading model from /Users/enke/.deeppavlov/models/ner_rus_bert/model]
2019-09-16 23:35:13,468 From //anaconda/lib/python3.6/site-packages/deeppavlov/core/models/tf_model.py:55: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.



### Тоже не очень быстро, но с gpu можно будет сильно ускорить

In [58]:
%%time

ners = ner_model([train['text'][700]])[1]

CPU times: user 4.26 s, sys: 370 ms, total: 4.63 s
Wall time: 951 ms


In [130]:
tokens_per_batch = 100

def get_ner(text):
    tokens = text.split(' ')
    ners = []
    for i in range(math.ceil(len(tokens)/tokens_per_batch)):
        ners += ner_model([' '.join(tokens[i*tokens_per_batch:np.min([(i+1)*tokens_per_batch, len(tokens)])])])[1][0]
    ners = pd.DataFrame.from_dict(Counter(ners), orient='index').T    
    return(ners)    
    

In [136]:
ners = [get_ner(train['text'][i]) for i in range(train.shape[0])]
ners = pd.concat(ners).fillna(0)
ners.reset_index(drop=True, inplace=True)

### Сделаем фичи о частотной встречаемости NER-ов 

In [148]:
ners['tokens'] = np.sum(ners, axis=1)
for col in ners.columns[:-1]:
    ners[col+'_freq'] = ners[col]/ners['tokens']

    
train = pd.concat([train,ners], axis=1)
cols_ners = ners.columns

In [180]:
run_xgb('Label_4', cols_ners)

[0]	test-auc:0.868255	train-auc:0.895766
[1]	test-auc:0.869762	train-auc:0.895884
[2]	test-auc:0.912056	train-auc:0.937064
[3]	test-auc:0.911911	train-auc:0.937064
[4]	test-auc:0.916772	train-auc:0.940184
[5]	test-auc:0.917647	train-auc:0.943774
[6]	test-auc:0.917064	train-auc:0.943438
[7]	test-auc:0.946038	train-auc:0.96563
[8]	test-auc:0.948128	train-auc:0.966922
[9]	test-auc:0.952115	train-auc:0.971384
[10]	test-auc:0.950948	train-auc:0.972457
[11]	test-auc:0.950754	train-auc:0.972222
[12]	test-auc:0.96106	train-auc:0.978345
[13]	test-auc:0.963102	train-auc:0.980257
[14]	test-auc:0.9614	train-auc:0.982421
[15]	test-auc:0.959553	train-auc:0.982253
[16]	test-auc:0.958726	train-auc:0.981851
[17]	test-auc:0.95999	train-auc:0.983092
[18]	test-auc:0.960087	train-auc:0.98279
[19]	test-auc:0.95931	train-auc:0.983394
[20]	test-auc:0.955955	train-auc:0.986346
[21]	test-auc:0.955323	train-auc:0.986715
[22]	test-auc:0.954934	train-auc:0.986279
[23]	test-auc:0.957365	train-auc:0.988292
[24]	test

In [181]:
run_xgb('Label_5', cols_ners)

[0]	test-auc:0.975121	train-auc:0.97774
[1]	test-auc:0.975121	train-auc:0.97774
[2]	test-auc:0.975054	train-auc:0.987657
[3]	test-auc:0.975054	train-auc:0.987657
[4]	test-auc:0.975054	train-auc:0.987657
[5]	test-auc:0.975054	train-auc:0.987657
[6]	test-auc:0.975054	train-auc:0.987633
[7]	test-auc:0.975054	train-auc:0.987633
[8]	test-auc:0.97411	train-auc:0.987728
[9]	test-auc:0.97411	train-auc:0.987728
[10]	test-auc:0.97384	train-auc:0.987538
[11]	test-auc:0.97384	train-auc:0.987538
[12]	test-auc:0.97384	train-auc:0.987538
[13]	test-auc:0.97384	train-auc:0.987776
[14]	test-auc:0.97384	train-auc:0.987776
[15]	test-auc:0.97384	train-auc:0.987776
[16]	test-auc:0.97384	train-auc:0.987776
[17]	test-auc:0.97384	train-auc:0.987776
[18]	test-auc:0.97384	train-auc:0.987776
[19]	test-auc:0.97384	train-auc:0.987776
[20]	test-auc:0.971278	train-auc:0.99717
[21]	test-auc:0.971143	train-auc:0.997122
[22]	test-auc:0.968649	train-auc:0.997265
[23]	test-auc:0.968447	train-auc:0.99717
[24]	test-auc:0.96

In [182]:
run_xgb('Label_6', cols_ners)

[0]	test-auc:0.532468	train-auc:0.53529
[1]	test-auc:0.570838	train-auc:0.569605
[2]	test-auc:0.570838	train-auc:0.569605
[3]	test-auc:0.572019	train-auc:0.569442
[4]	test-auc:0.570838	train-auc:0.56684
[5]	test-auc:0.575167	train-auc:0.690112
[6]	test-auc:0.577529	train-auc:0.693202
[7]	test-auc:0.577529	train-auc:0.693202
[8]	test-auc:0.577529	train-auc:0.693202
[9]	test-auc:0.577529	train-auc:0.693202
[10]	test-auc:0.577529	train-auc:0.69304
[11]	test-auc:0.695396	train-auc:0.722638
[12]	test-auc:0.695396	train-auc:0.722313
[13]	test-auc:0.695986	train-auc:0.722313
[14]	test-auc:0.695986	train-auc:0.722394
[15]	test-auc:0.638331	train-auc:0.837047
[16]	test-auc:0.756002	train-auc:0.909497
[17]	test-auc:0.765053	train-auc:0.932672
[18]	test-auc:0.77076	train-auc:0.946821
[19]	test-auc:0.754624	train-auc:0.957147
[20]	test-auc:0.745179	train-auc:0.964059
[21]	test-auc:0.733373	train-auc:0.966499
[22]	test-auc:0.735144	train-auc:0.975443
[23]	test-auc:0.734553	train-auc:0.983412
[24]	t

In [183]:
run_xgb('Label_3', cols_ners)

[0]	test-auc:0.5	train-auc:0.5
[1]	test-auc:0.5	train-auc:0.5
[2]	test-auc:0.5	train-auc:0.5
[3]	test-auc:0.5	train-auc:0.5
[4]	test-auc:0.5	train-auc:0.5
[5]	test-auc:0.5	train-auc:0.5
[6]	test-auc:0.5	train-auc:0.5
[7]	test-auc:0.966455	train-auc:0.995685
[8]	test-auc:0.966808	train-auc:0.997646
[9]	test-auc:0.966808	train-auc:0.997646
[10]	test-auc:0.966808	train-auc:0.997646
[11]	test-auc:0.966808	train-auc:0.997646
[12]	test-auc:0.966808	train-auc:0.997646
[13]	test-auc:0.966808	train-auc:0.997646
[14]	test-auc:0.966808	train-auc:0.997646
[15]	test-auc:0.959746	train-auc:0.999085
[16]	test-auc:0.959746	train-auc:0.999085
[17]	test-auc:0.959746	train-auc:0.999085
[18]	test-auc:0.959746	train-auc:0.999085
[19]	test-auc:0.959746	train-auc:0.999085
[20]	test-auc:0.959746	train-auc:0.999085
[21]	test-auc:0.959746	train-auc:0.999085
[22]	test-auc:0.959746	train-auc:0.999085
[23]	test-auc:0.959746	train-auc:0.999085
[24]	test-auc:0.971045	train-auc:0.999085
[25]	test-auc:0.971045	train-a

### В целом работает хорошо, но для одного класса проваливается по сравнению с tf-idf

In [179]:
########### text_structure ### tfidf ### NER ###
### Label_4     0.912          0.996    0.961
### Label_5     0.991          0.993    0.977
### Label_6     0.742          0.982    0.813
### Label_3     0.893          0.908    0.985

### Попробуем к фичам из NER добавить фичи о структуре текста

In [191]:
run_xgb('Label_4', list(cols_ners)+cols)

[0]	test-auc:0.847885	train-auc:0.871327
[1]	test-auc:0.868935	train-auc:0.905009
[2]	test-auc:0.903938	train-auc:0.935571
[3]	test-auc:0.911035	train-auc:0.939026
[4]	test-auc:0.912931	train-auc:0.940855
[5]	test-auc:0.918911	train-auc:0.942683
[6]	test-auc:0.921536	train-auc:0.947229
[7]	test-auc:0.939135	train-auc:0.96615
[8]	test-auc:0.948031	train-auc:0.972491
[9]	test-auc:0.9456	train-auc:0.973044
[10]	test-auc:0.950024	train-auc:0.977909
[11]	test-auc:0.94983	train-auc:0.977707
[12]	test-auc:0.950024	train-auc:0.978647
[13]	test-auc:0.951094	train-auc:0.981163
[14]	test-auc:0.953184	train-auc:0.982505
[15]	test-auc:0.949392	train-auc:0.983343
[16]	test-auc:0.951288	train-auc:0.984652
[17]	test-auc:0.951142	train-auc:0.985893
[18]	test-auc:0.951531	train-auc:0.986732
[19]	test-auc:0.953427	train-auc:0.98799
[20]	test-auc:0.956928	train-auc:0.990372
[21]	test-auc:0.957705	train-auc:0.991479
[22]	test-auc:0.959067	train-auc:0.993022
[23]	test-auc:0.956393	train-auc:0.994062
[24]	te

In [192]:
run_xgb('Label_5', list(cols_ners)+cols)

[0]	test-auc:0.975121	train-auc:0.97774
[1]	test-auc:0.975121	train-auc:0.97774
[2]	test-auc:0.975054	train-auc:0.987657
[3]	test-auc:0.975054	train-auc:0.987657
[4]	test-auc:0.975054	train-auc:0.987657
[5]	test-auc:0.975054	train-auc:0.987657
[6]	test-auc:0.975054	train-auc:0.987633
[7]	test-auc:0.975054	train-auc:0.987633
[8]	test-auc:0.97411	train-auc:0.987728
[9]	test-auc:0.97411	train-auc:0.987728
[10]	test-auc:0.97384	train-auc:0.987538
[11]	test-auc:0.97384	train-auc:0.987538
[12]	test-auc:0.97384	train-auc:0.987538
[13]	test-auc:0.973571	train-auc:0.997289
[14]	test-auc:0.973571	train-auc:0.997527
[15]	test-auc:0.973099	train-auc:0.997455
[16]	test-auc:0.973099	train-auc:0.997598
[17]	test-auc:0.973099	train-auc:0.997455
[18]	test-auc:0.975728	train-auc:0.997931
[19]	test-auc:0.976335	train-auc:0.997931
[20]	test-auc:0.976739	train-auc:0.997931
[21]	test-auc:0.976605	train-auc:0.998026
[22]	test-auc:0.976739	train-auc:0.998121
[23]	test-auc:0.976739	train-auc:0.998311
[24]	test

In [193]:
run_xgb('Label_6', list(cols_ners)+cols)

[0]	test-auc:0.532468	train-auc:0.53529
[1]	test-auc:0.55549	train-auc:0.633599
[2]	test-auc:0.55549	train-auc:0.633599
[3]	test-auc:0.552145	train-auc:0.632542
[4]	test-auc:0.553522	train-auc:0.632867
[5]	test-auc:0.553916	train-auc:0.632867
[6]	test-auc:0.553916	train-auc:0.632867
[7]	test-auc:0.553325	train-auc:0.630834
[8]	test-auc:0.553325	train-auc:0.630834
[9]	test-auc:0.636364	train-auc:0.786876
[10]	test-auc:0.643447	train-auc:0.795333
[11]	test-auc:0.675718	train-auc:0.878598
[12]	test-auc:0.681621	train-auc:0.888437
[13]	test-auc:0.680834	train-auc:0.889738
[14]	test-auc:0.684573	train-auc:0.891039
[15]	test-auc:0.687131	train-auc:0.892991
[16]	test-auc:0.682605	train-auc:0.893316
[17]	test-auc:0.683392	train-auc:0.89421
[18]	test-auc:0.685163	train-auc:0.893966
[19]	test-auc:0.701102	train-auc:0.949341
[20]	test-auc:0.700315	train-auc:0.948366
[21]	test-auc:0.712712	train-auc:0.961538
[22]	test-auc:0.713499	train-auc:0.969914
[23]	test-auc:0.712318	train-auc:0.969101
[24]	t

In [194]:
run_xgb('Label_3', list(cols_ners)+cols)

[0]	test-auc:0.495763	train-auc:0.559362
[1]	test-auc:0.495763	train-auc:0.559362
[2]	test-auc:0.495763	train-auc:0.559362
[3]	test-auc:0.812147	train-auc:0.857087
[4]	test-auc:0.812147	train-auc:0.857087
[5]	test-auc:0.812147	train-auc:0.857087
[6]	test-auc:0.812147	train-auc:0.857087
[7]	test-auc:0.811441	train-auc:0.855779
[8]	test-auc:0.893715	train-auc:0.99307
[9]	test-auc:0.893008	train-auc:0.991501
[10]	test-auc:0.893715	train-auc:0.994116
[11]	test-auc:0.987288	train-auc:0.996339
[12]	test-auc:0.991525	train-auc:0.996339
[13]	test-auc:0.991525	train-auc:0.996862
[14]	test-auc:0.992232	train-auc:0.996862
[15]	test-auc:0.992232	train-auc:0.998431
[16]	test-auc:0.992232	train-auc:0.998954
[17]	test-auc:0.992232	train-auc:0.998954
[18]	test-auc:0.991525	train-auc:0.999215
[19]	test-auc:0.991525	train-auc:0.999215
[20]	test-auc:0.992232	train-auc:0.998954
[21]	test-auc:0.991525	train-auc:0.999215
[22]	test-auc:0.991525	train-auc:0.999215
[23]	test-auc:0.990819	train-auc:0.999215
[24

### Лучше не стало

In [ ]:
########### text_structure ### tfidf ### NER ### NER+text_structure ###
### Label_4     0.912          0.996    0.961        0.959
### Label_5     0.991          0.993    0.977        0.989
### Label_6     0.742          0.982    0.813        0.805
### Label_3     0.893          0.908    0.985        0.992

### Дальше хотелось бы сделать SOTA эмбеддинги для слов, но даже в DeepPavlov не нашлось готовых ни для BERT, ни для XLNet. Но есть ELMo, тоже неплохо, берем его

In [212]:
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder

In [219]:
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

### Без gpu считается совсем не быстро

In [240]:
%%time

elmo([re.findall('([a-zA-Zа-яА-Я]+)', train['text'][700])])

CPU times: user 7.82 s, sys: 322 ms, total: 8.14 s
Wall time: 3.72 s


array([[-0.3459992 ,  0.02534257,  0.46729198, ...,  0.08188066,
         0.31075752,  0.739845  ]], dtype=float32)

In [243]:
def get_elmo(text):
    return(elmo([re.findall('([a-zA-Zа-яА-Я]+)', text)]))

In [245]:
elmo_emb = [get_elmo(train['text'][i]) for i in range(train.shape[0])]
elmo_cols = ['elmo_'+str(i) for i in range(elmo_emb[0].shape[1])]
elmo_emb = pd.DataFrame(np.vstack(elmo_emb), columns=elmo_cols)

train = pd.concat([train, elmo_emb],axis=1)

In [262]:
run_xgb('Label_4', elmo_cols)

[0]	test-auc:0.815605	train-auc:0.927486
[1]	test-auc:0.881283	train-auc:0.946877
[2]	test-auc:0.879144	train-auc:0.960044
[3]	test-auc:0.90282	train-auc:0.966653
[4]	test-auc:0.909772	train-auc:0.981196
[5]	test-auc:0.953281	train-auc:0.98903
[6]	test-auc:0.959893	train-auc:0.997517
[7]	test-auc:0.960039	train-auc:0.998759
[8]	test-auc:0.962227	train-auc:0.999228
[9]	test-auc:0.960379	train-auc:0.999463
[10]	test-auc:0.960574	train-auc:0.999665
[11]	test-auc:0.963247	train-auc:0.999933
[12]	test-auc:0.963053	train-auc:1
[13]	test-auc:0.963442	train-auc:1
[14]	test-auc:0.963247	train-auc:1
[15]	test-auc:0.963539	train-auc:1
[16]	test-auc:0.964317	train-auc:1
[17]	test-auc:0.964317	train-auc:1
[18]	test-auc:0.96315	train-auc:1
[19]	test-auc:0.961692	train-auc:1
[20]	test-auc:0.961692	train-auc:1
[21]	test-auc:0.962081	train-auc:1
[22]	test-auc:0.961497	train-auc:1
[23]	test-auc:0.960719	train-auc:1
[24]	test-auc:0.959067	train-auc:1
[25]	test-auc:0.958872	train-auc:1
[26]	test-auc:0.958

In [263]:
run_xgb('Label_5', elmo_cols)

[0]	test-auc:0.920173	train-auc:0.988085
[1]	test-auc:0.920173	train-auc:0.988085
[2]	test-auc:0.949029	train-auc:0.998858
[3]	test-auc:0.94822	train-auc:0.999191
[4]	test-auc:0.949973	train-auc:0.999524
[5]	test-auc:0.949299	train-auc:0.999619
[6]	test-auc:0.949231	train-auc:0.999715
[7]	test-auc:0.947074	train-auc:0.999857
[8]	test-auc:0.959008	train-auc:0.999857
[9]	test-auc:0.958738	train-auc:1
[10]	test-auc:0.960289	train-auc:1
[11]	test-auc:0.959614	train-auc:1
[12]	test-auc:0.959547	train-auc:1
[13]	test-auc:0.959412	train-auc:1
[14]	test-auc:0.959412	train-auc:1
[15]	test-auc:0.959008	train-auc:1
[16]	test-auc:0.958064	train-auc:1
[17]	test-auc:0.968716	train-auc:1
[18]	test-auc:0.971211	train-auc:1
[19]	test-auc:0.97175	train-auc:1
[20]	test-auc:0.971953	train-auc:1
[21]	test-auc:0.973301	train-auc:1
[22]	test-auc:0.973301	train-auc:1
[23]	test-auc:0.973301	train-auc:1
[24]	test-auc:0.974515	train-auc:1
[25]	test-auc:0.974852	train-auc:1
[26]	test-auc:0.974852	train-auc:1
[27]

In [264]:
run_xgb('Label_6', elmo_cols)

[0]	test-auc:0.714679	train-auc:0.956253
[1]	test-auc:0.826249	train-auc:0.995121
[2]	test-auc:0.821921	train-auc:0.992682
[3]	test-auc:0.823495	train-auc:0.993495
[4]	test-auc:0.823101	train-auc:0.993495
[5]	test-auc:0.859701	train-auc:0.998211
[6]	test-auc:0.868359	train-auc:0.998699
[7]	test-auc:0.87013	train-auc:0.999187
[8]	test-auc:0.868949	train-auc:0.999187
[9]	test-auc:0.87013	train-auc:0.999187
[10]	test-auc:0.872098	train-auc:0.999187
[11]	test-auc:0.870917	train-auc:0.999675
[12]	test-auc:0.87013	train-auc:0.999675
[13]	test-auc:0.869736	train-auc:0.999675
[14]	test-auc:0.866588	train-auc:0.999837
[15]	test-auc:0.864227	train-auc:0.999837
[16]	test-auc:0.902401	train-auc:1
[17]	test-auc:0.901614	train-auc:1
[18]	test-auc:0.898072	train-auc:1
[19]	test-auc:0.902597	train-auc:1
[20]	test-auc:0.899843	train-auc:1
[21]	test-auc:0.922865	train-auc:1
[22]	test-auc:0.922078	train-auc:1
[23]	test-auc:0.921881	train-auc:1
[24]	test-auc:0.9207	train-auc:1
[25]	test-auc:0.924439	train

In [265]:
run_xgb('Label_3', elmo_cols)

[0]	test-auc:0.828743	train-auc:0.999608
[1]	test-auc:0.909958	train-auc:1
[2]	test-auc:0.907839	train-auc:1
[3]	test-auc:0.906427	train-auc:1
[4]	test-auc:0.907133	train-auc:1
[5]	test-auc:0.90678	train-auc:1
[6]	test-auc:0.90678	train-auc:1
[7]	test-auc:0.907839	train-auc:1
[8]	test-auc:0.907839	train-auc:1
[9]	test-auc:0.906073	train-auc:1
[10]	test-auc:0.905367	train-auc:1
[11]	test-auc:0.90572	train-auc:1
[12]	test-auc:0.905014	train-auc:1
[13]	test-auc:0.905014	train-auc:1
[14]	test-auc:0.905014	train-auc:1
[15]	test-auc:0.903602	train-auc:1
[16]	test-auc:0.903602	train-auc:1
[17]	test-auc:0.903602	train-auc:1
[18]	test-auc:0.903602	train-auc:1
[19]	test-auc:0.903602	train-auc:1
[20]	test-auc:0.902542	train-auc:1
[21]	test-auc:0.902542	train-auc:1
[22]	test-auc:0.90113	train-auc:1
[23]	test-auc:0.90113	train-auc:1
[24]	test-auc:0.90113	train-auc:1
[25]	test-auc:0.90113	train-auc:1
[26]	test-auc:0.90113	train-auc:1
[27]	test-auc:0.90113	train-auc:1
[28]	test-auc:0.897952	train-auc

### Весьма неплохой результат

In [ ]:
########### text_structure ### tfidf ### NER ### NER+text_structure ### ELMO ###
### Label_4     0.912          0.996    0.961        0.959             0.966
### Label_5     0.991          0.993    0.977        0.989             0.985
### Label_6     0.742          0.982    0.813        0.805             0.960
### Label_3     0.893          0.908    0.985        0.992             0.957

### Пока по качеству очень неплох tf-idf, но лемматизация долго считается. Попробуем сделать tf-idf на стемминге

In [360]:
from nltk.stem import SnowballStemmer

In [361]:
stemmer = SnowballStemmer("russian")

In [362]:
def stem_words(text):
    tokens = re.findall('([a-zA-Zа-яА-Я]{3,})', text)
    tokens = [i for i in tokens if not i in nltk.corpus.stopwords.words('russian')]
    tokens = ' '.join([stemmer.stem(token) for token in tokens])
    return(tokens)

### Вот такая предобработка текста уже заметно быстрее

In [332]:
%%time
stem_words(train['text'][700])

CPU times: user 48.2 ms, sys: 8.62 ms, total: 56.8 ms
Wall time: 59.8 ms


'вопрос повестк дня проект решен избра совет директор обществ след состав анипкин макс николаевич антонюк алекс владимирович быстр евген александрович вейалайн яркк армас есик александр юрьевич каплун павел сергеевич копон харр ээрик ушк александр анатольевич чумаченк натал викторовн итог голосован четверт вопрос повестк дня годов общ собран акционер числ голос котор облада лиц включен список лиц имевш прав участ общ собран дан вопрос повестк дня общ собран числ голос приход голос акц обществ дан вопрос повестк дня общ собран определен учет положен пункт положен числ голос котор облада лиц приня участ общ собран дан вопрос повестк дня общ собран кворум дан вопрос имел кандидат числ голос отда кажд вариант голосован распределен голос кандидат быстр евген александрович чумаченк натал викторовн есик александр юрьевич анипкин макс николаевич каплун павел сергеевич антонюк алекс владимирович ушк александр анатольевич вейалайн яркк армас копон харр ээрик эт воздержа числ голос котор подсчиты

In [363]:
train['text_stem'] = [stem_words(train['text'][i]) for i in range(train.shape[0])]

### Так как слова не приводятся к полностью стандартному виду, топовых слов придется оставить побольше, чтобы ничего не потерять, - 400 например

In [364]:
tfidf_feautures = 400

vectorizer = TfidfVectorizer(max_features=tfidf_feautures, ngram_range=(1, 2))

tfidf_cols = ['tfidf_stem_'+str(i) for i in range(tfidf_feautures)]

vectorizer = vectorizer.fit(train.ix[train_index]['text_stem'])
tfidf = vectorizer.transform(train['text_stem'])
tfidf = pd.DataFrame(tfidf.todense(), columns=tfidf_cols)

train = pd.concat([train, tfidf], axis=1)

In [345]:
run_xgb('Label_4', tfidf_cols)

[0]	test-auc:0.948031	train-auc:0.998021
[1]	test-auc:0.951142	train-auc:0.998692
[2]	test-auc:0.951726	train-auc:0.998759
[3]	test-auc:0.950267	train-auc:0.999161
[4]	test-auc:0.952795	train-auc:0.999446
[5]	test-auc:0.951823	train-auc:0.999446
[6]	test-auc:0.962859	train-auc:0.999312
[7]	test-auc:0.962129	train-auc:0.999681
[8]	test-auc:0.961692	train-auc:0.999648
[9]	test-auc:0.96247	train-auc:0.999614
[10]	test-auc:0.98177	train-auc:0.999581
[11]	test-auc:0.982742	train-auc:0.999748
[12]	test-auc:0.982645	train-auc:0.999782
[13]	test-auc:0.982839	train-auc:0.999748
[14]	test-auc:0.983957	train-auc:0.999681
[15]	test-auc:0.98561	train-auc:0.999681
[16]	test-auc:0.986388	train-auc:0.999748
[17]	test-auc:0.986291	train-auc:0.999849
[18]	test-auc:0.985561	train-auc:0.999815
[19]	test-auc:0.985805	train-auc:0.999849
[20]	test-auc:0.991444	train-auc:0.999866
[21]	test-auc:0.991152	train-auc:0.999866
[22]	test-auc:0.991152	train-auc:0.999899
[23]	test-auc:0.99368	train-auc:0.999966
[24]	t

In [346]:
run_xgb('Label_5', tfidf_cols)

[0]	test-auc:0.942219	train-auc:0.977906
[1]	test-auc:0.941141	train-auc:0.987823
[2]	test-auc:0.952198	train-auc:0.997455
[3]	test-auc:0.952198	train-auc:0.997598
[4]	test-auc:0.952468	train-auc:0.998621
[5]	test-auc:0.951996	train-auc:0.998383
[6]	test-auc:0.981594	train-auc:0.999334
[7]	test-auc:0.982807	train-auc:0.999477
[8]	test-auc:0.983077	train-auc:0.999667
[9]	test-auc:0.983077	train-auc:0.999572
[10]	test-auc:0.98301	train-auc:0.999715
[11]	test-auc:0.982942	train-auc:0.999715
[12]	test-auc:0.98274	train-auc:0.99981
[13]	test-auc:0.982403	train-auc:0.999715
[14]	test-auc:0.982403	train-auc:0.99981
[15]	test-auc:0.982403	train-auc:0.99981
[16]	test-auc:0.982403	train-auc:0.99981
[17]	test-auc:0.981122	train-auc:0.999952
[18]	test-auc:0.98092	train-auc:0.999905
[19]	test-auc:0.980785	train-auc:0.999952
[20]	test-auc:0.980717	train-auc:0.999952
[21]	test-auc:0.980515	train-auc:0.999952
[22]	test-auc:0.980313	train-auc:0.999952
[23]	test-auc:0.980313	train-auc:0.999952
[24]	test

In [347]:
run_xgb('Label_6', tfidf_cols)

[0]	test-auc:0.989177	train-auc:0.996991
[1]	test-auc:0.991932	train-auc:0.997805
[2]	test-auc:0.991932	train-auc:0.997967
[3]	test-auc:0.996261	train-auc:0.999837
[4]	test-auc:0.997442	train-auc:0.999837
[5]	test-auc:0.998623	train-auc:0.999837
[6]	test-auc:0.998623	train-auc:0.999837
[7]	test-auc:0.998623	train-auc:0.999837
[8]	test-auc:0.998623	train-auc:0.999837
[9]	test-auc:0.998819	train-auc:1
[10]	test-auc:0.998819	train-auc:1
[11]	test-auc:0.998819	train-auc:1
[12]	test-auc:0.998819	train-auc:1
[13]	test-auc:0.998819	train-auc:1
[14]	test-auc:0.998819	train-auc:1
[15]	test-auc:0.998819	train-auc:1
[16]	test-auc:0.998819	train-auc:1
[17]	test-auc:0.998819	train-auc:1
[18]	test-auc:0.998819	train-auc:1
[19]	test-auc:0.998819	train-auc:1
[20]	test-auc:0.998819	train-auc:1
[21]	test-auc:0.998819	train-auc:1
[22]	test-auc:0.998819	train-auc:1
[23]	test-auc:0.998819	train-auc:1
[24]	test-auc:0.998819	train-auc:1
[25]	test-auc:0.998819	train-auc:1
[26]	test-auc:0.998819	train-auc:1
[2

In [348]:
run_xgb('Label_3', tfidf_cols)

[0]	test-auc:0.820621	train-auc:0.99647
[1]	test-auc:0.984816	train-auc:0.997254
[2]	test-auc:0.984816	train-auc:0.997123
[3]	test-auc:0.989054	train-auc:0.997646
[4]	test-auc:0.987641	train-auc:0.997646
[5]	test-auc:0.990466	train-auc:0.997908
[6]	test-auc:0.991172	train-auc:0.998431
[7]	test-auc:0.991172	train-auc:0.998692
[8]	test-auc:0.992585	train-auc:0.998692
[9]	test-auc:0.991172	train-auc:0.998954
[10]	test-auc:0.991172	train-auc:0.998954
[11]	test-auc:0.990466	train-auc:0.999477
[12]	test-auc:0.991172	train-auc:0.999739
[13]	test-auc:0.991879	train-auc:0.999739
[14]	test-auc:0.991172	train-auc:0.999739
[15]	test-auc:0.993291	train-auc:0.999739
[16]	test-auc:0.993291	train-auc:0.999739
[17]	test-auc:0.993291	train-auc:0.999739
[18]	test-auc:0.992585	train-auc:1
[19]	test-auc:0.993291	train-auc:1
[20]	test-auc:0.993291	train-auc:1
[21]	test-auc:0.992585	train-auc:1
[22]	test-auc:0.98976	train-auc:1
[23]	test-auc:0.992585	train-auc:1
[24]	test-auc:0.994703	train-auc:1
[25]	test-a

### Результаты не хуже, чем при лемматизации, для одного из лейблов даже лучше, но это возможно шум

In [336]:
########### text_structure ### tfidf(lemma) ### NER ### NER+text_structure ### ELMO ### tfidf(stem) ###
### Label_4     0.912           0.996          0.961        0.959             0.966       0.997
### Label_5     0.991           0.993          0.977        0.989             0.985       0.990
### Label_6     0.742           0.982          0.813        0.805             0.960       0.998
### Label_3     0.893           0.908          0.985        0.992             0.957       0.997

### Можно было бы попробовать другие эмбеддинги, или дообучить ELMo, при наличии большего количества данных. Но последняя версия модели на tf-idf фичах быстра и дает отличные результаты, поэтому остановимся на ней

### Для финального обучения моделей хорошо бы обучать их на сете для обучения, тюня параметры для него на валидации. Но данных мало, а некоторых классов просто критически мало, придется обучить модели с теми же гиперпараметрами на всем трейновом сете (в xgboost все равно небольшие изменения параметров практически не влияют на качество)

### Сделаем преобразование tf-idf на все трейновом датасете

In [444]:
for col in tfidf_cols:
    del train[col]

In [445]:
vectorizer = TfidfVectorizer(max_features=tfidf_feautures, ngram_range=(1, 2))

tfidf = vectorizer.fit_transform(train['text_stem'])
tfidf = pd.DataFrame(tfidf.todense(), columns=tfidf_cols)

train = pd.concat([train, tfidf], axis=1)

### Сделаем фичи для тестового сета

In [438]:
test['text_stem'] = [stem_words(test['text'][i]) for i in range(test.shape[0])]

In [447]:
tfidf = vectorizer.transform(test['text_stem'])
tfidf = pd.DataFrame(tfidf.todense(), columns=tfidf_cols)

test = pd.concat([test, tfidf], axis=1)

### Предсказания для тестового сета

In [372]:
n_parallel = 7

param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'auc'
param['nthread'] = n_parallel

param['eta'] = 0.1
param['max_depth'] = 4
param['min_child_weight'] = 1
param['subsample'] = 1
param['colsample_bytree'] = 1
param['colsample_bylevel'] = 1
param['lambda'] = 1
param['alpha'] = 0
param['seed'] = 0
param['gamma'] = 0

numround = {'Label_1':20, 'Label_2':30, 'Label_3':30, 'Label_4':30, 'Label_5':10, 'Label_6':10}

In [373]:
def fit_xgb(label, cols):
    dtrain = xgb.DMatrix(train[cols], label=train[label])
    model = xgb.train(param, dtrain, numround[label])
    return(model)

In [378]:
def predict_xgb(label, cols):
    model = fit_xgb(label, cols)
    dtest = xgb.DMatrix(test[cols])
    pred = model.predict(dtest)
    return(pred)   

In [448]:
predictions={}
for label in labels:
    predictions[label]=predict_xgb(label, tfidf_cols)   

### Проверим, что при преобразовании вероятностей к меткам класса, пропорции классов в трейне и тесте приблизительно сохраняются

In [449]:
threshold = 0.5
[[label, np.mean(predictions[label]>threshold)] for label in labels]

[['Label_1', 0.0],
 ['Label_2', 0.0],
 ['Label_3', 0.0],
 ['Label_4', 0.85],
 ['Label_5', 0.1],
 ['Label_6', 0.03]]

In [450]:
np.sum(train[labels], axis=0)/train.shape[0]

Label_1    0.008242
Label_2    0.010989
Label_3    0.019231
Label_4    0.825549
Label_5    0.115385
Label_6    0.032967
dtype: float64

### Сохраним предсказания в виде вероятностей и меток класса

In [451]:
predictions=pd.DataFrame.from_dict(predictions)

predictions.to_csv(data_dir+'predictions_prob.csv',index=False,sep=';')
(predictions>threshold).astype('int').to_csv(data_dir+'predictions_label.csv',index=False,sep=';')